# 🎯 HaptiMed : Évaluation de l'Expertise Chirurgicale (Steering Task)

Ce projet de recherche vise à discriminer le niveau d'expertise (Novices vs Experts) en comparant les scores d'évaluation clinique standards (OSATS) aux performances biomécaniques quantitatives. 

Pour ce faire, nous utilisons une tâche de pilotage circulaire (Steering Task) modélisée selon la **loi d'Accot et Zhai** (dérivée de la loi de Fitts pour la navigation continue), en y ajoutant des contraintes haptiques.

## 🧪 Paradigme Expérimental

Le participant doit naviguer à l'intérieur d'un tunnel circulaire. L'expérience croise 4 conditions expérimentales :

1. **VP (Vitesse - Précision) avec feedback** : Tracé classique avec aide visuelle.
2. **VP (Vitesse - Précision) sans feedback** : Tracé basé sur la proprioception (seul le pointeur est visible).
3. **FVP (Force - Vitesse - Précision) avec feedback** : Maintien d'une force cible (MVC) avec retour visuel par jauge de couleur.
4. **FVP (Force - Vitesse - Précision) sans feedback** : Calibrage initial puis maintien de la force de mémoire tout au long de la trajectoire.

## 📂 Architecture du Projet

Le dépôt est organisé de manière modulaire pour garantir la reproductibilité de la recherche :

* 📁 **`python/`** : Scripts sources (Pipeline complet).
  * *Acquisition :* `steering_task.py`, `calibration_mvc.py`
  * *Traitement :* `process_data.py` (Filtrage Butterworth, calcul de cinématique et fluidité).
  * *Analyse :* `analysis_global.py`, `analysis_publication.py`, `analysis_ml.py`
  * *Génération de rapports :* `generate_tech_doc.py`, `generate_master_report.py`
* 📁 **`data/`** : Contient le dictionnaire `metadata.csv` (scores OSATS). *(Note : Les données brutes des participants sont ignorées via `.gitignore` pour des raisons d'éthique et de RGPD)*.
* 📁 **`doc/`** : Contient les graphiques de résultats et tableaux de score.
* 📓 **`Analyse_Demonstration.ipynb`** : Le carnet de notes interactif démontrant la logique d'analyse.
* 📑 **`Protocole_HaptiMed.html`** & **`Documentation_Technique.html`** : La documentation technique complète et le protocole de passation détaillé (consignes exactes pour les participants), accessibles directement à la racine.


## 🚀 Comment lire ce projet ?

Pour comprendre la démarche d'analyse liant les scores OSATS aux performances de la Steering Task (sans avoir à exécuter tout le pipeline), **ouvrez le fichier `Analyse_Demonstration.ipynb`** situé à la racine du projet. Ce notebook mêle explications, code Python et graphiques pré-générés.

## ⚙️ Exécution en local (Chercheurs)

Pour lancer l'environnement complet :
1. Activer l'environnement virtuel : `source venv/bin/activate` (ou `venv\Scripts\activate` sur Windows).
2. Lancer la tâche de passation : `python python/steering_task.py`.
3. Traiter les trajectoires : `python python/process_data.py`.

# 🎯 Démonstration de l'Analyse HaptiMed
**Objectif :** Discriminer le niveau d'expertise chirurgicale (Novices vs Experts) en croisant les scores cliniques OSATS avec les performances issues d'une tâche de pilotage circulaire (Loi d'Accot-Zhai).

Ce notebook présente de manière synthétique comment les données traitées (Issues de `process_data.py`) permettent de valider nos hypothèses expérimentales.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Configuration esthétique
sns.set_theme(style="whitegrid")

# 1. Chargement du dataset consolidé (Généré par le pipeline en amont)
# (Assurez-vous d'avoir un fichier test valide dans data/features/)
try:
    df = pd.read_csv('data/features/dataset_features.csv')
    df_valide = df[df['Group'].isin(['Novice', 'Expert'])].dropna(subset=['OSATS', 'Throughput_ISO'])
    
    print("Aperçu des données chargées :")
    display(df_valide[['ID', 'Group', 'Condition', 'OSATS', 'Throughput_ISO', 'LDLJ']].head())
except FileNotFoundError:
    print("⚠️ Fichier introuvable. Lancez le script python/process_data.py pour générer les features.")

⚠️ Fichier introuvable. Lancez le script python/process_data.py pour générer les features.


## Loi d'Accot-Zhai et Validité Concourante (OSATS)
La loi d'Accot-Zhai stipule que le temps de mouvement dans un tunnel contraint dépend de sa longueur et de sa largeur. Nous utilisons le **Throughput (TP)** pour quantifier cette efficience motrice.
Vérifions si le score clinique OSATS est bien corrélé à la performance biomécanique (TP) mesurée par notre tâche.

In [2]:
if 'df_valide' in locals() and not df_valide.empty:
    # On fait la moyenne par participant pour avoir un score global
    df_subj = df_valide.groupby(['ID', 'Group'])[['OSATS', 'Throughput_ISO']].mean().reset_index()

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='OSATS', y='Throughput_ISO', hue='Group', palette=['#3498db', '#e74c3c'], data=df_subj, s=100)
    sns.regplot(x='OSATS', y='Throughput_ISO', data=df_subj, scatter=False, color='gray', line_kws={'linestyle':'--'})
    
    plt.title("Corrélation entre l'Évaluation Clinique (OSATS) et l'Efficience Motrice (Accot-Zhai)", pad=15)
    plt.xlabel("Score Global OSATS")
    plt.ylabel("Throughput Effectif (bits/s)")
    plt.show()
else:
    print("Données insuffisantes pour générer le graphique.")

Données insuffisantes pour générer le graphique.


*L'ensemble des tests statistiques formels (t-tests, Cohen's d) et des modèles de Machine Learning se trouvent respectivement dans les scripts `analysis_publication.py` et `analysis_ml.py` du dossier `/python`.*